In [ ]:
import os
import pickle
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import warnings
import datetime as dt
import yfinance as yf
import Function_model

# Keras / TensorFlow imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

rcParams['figure.figsize'] = 12,6
warnings.filterwarnings('ignore')

# Chọn mã cổ phiếu phù hợp với tên notebook
bank_stock_name = "VCB.VN"

# Thư mục lưu model
model_dir = os.path.join(os.getcwd(), '..', 'Save_model')
stock_model_dir = os.path.join(model_dir, bank_stock_name)
print('Model dir:', stock_model_dir)
if not os.path.exists(stock_model_dir):
    os.makedirs(stock_model_dir, exist_ok=True)


Model dir: c:\Users\Dell\Documents\BIG_DATA_NHOM\Train_model\..\Save_model\VCB.VN


In [4]:
stock = pd.read_csv(f'../data/{bank_stock_name}.csv')
stock.head()

,Price,Date,Close,High,Low,Open,Volume
0,0,2020-11-26,41421.667969,41421.667969,40804.749147,41201.338710,1573311
1,1,2020-11-27,41641.992188,41641.992188,41069.142335,41421.666813,2727313
2,2,2020-11-30,40981.007812,41421.666312,40320.023941,41289.467211,3344066
3,3,2020-12-01,40540.355469,41069.144147,39967.501713,39967.501713,2618476
4,4,2020-12-02,41201.339844,41201.339844,40320.026662,40540.355927,1682259


# Xây dựng mô hình LSTM

In [8]:
stock.index=stock['Date']

stock.sort_index(ascending=True, axis=0)

stock

,Price,Date,Close,High,Low,Open,Volume
Date,,,,,,,
2020-11-26,0,2020-11-26,41421.667969,41421.667969,40804.749147,41201.338710,1573311
2020-11-27,1,2020-11-27,41641.992188,41641.992188,41069.142335,41421.666813,2727313
2020-11-30,2,2020-11-30,40981.007812,41421.666312,40320.023941,41289.467211,3344066
2020-12-01,3,2020-12-01,40540.355469,41069.144147,39967.501713,39967.501713,2618476
2020-12-02,4,2020-12-02,41201.339844,41201.339844,40320.026662,40540.355927,1682259
...,...,...,...,...,...,...,...
2025-11-19,1242,2025-11-19,59400.000000,60100.000000,59300.000000,59900.000000,3203759
2025-11-20,1243,2025-11-20,59400.000000,59700.000000,59200.000000,59600.000000,2737705
2025-11-21,1244,2025-11-21,59000.000000,59200.000000,58700.000000,59100.000000,4127425


In [9]:
PAST_RANGE = 80

#creating dataframe
stock_lstm = pd.DataFrame(index=range(0,len(stock)),columns=['Date', 'Close'])
for i in range(0,len(stock)):
    stock_lstm['Date'][i] = stock['Date'][i]
    stock_lstm['Close'][i] = stock['Close'][i]

#setting index
stock_lstm.index = stock_lstm.Date
stock_lstm.drop('Date', axis=1, inplace=True)


In [15]:
#creating train and test sets
dataset = stock_lstm.values

train = dataset[0:990,:]
valid = dataset[990:,:]



In [11]:
#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

x_train, y_train = [], []
for i in range(PAST_RANGE,len(train)):
    x_train.append(scaled_data[i-PAST_RANGE:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))


In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)
print('Fitting Model')

# Predicting values using past data from the train data
inputs = stock_lstm[len(stock_lstm) - len(valid) - PAST_RANGE:].values
inputs = inputs.reshape(-1,1)
inputs = scaler.transform(inputs)

X_test = []
for i in range(PAST_RANGE, inputs.shape[0]):
    X_test.append(inputs[i-PAST_RANGE:i, 0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
closing_price = model.predict(X_test)
preds = scaler.inverse_transform(closing_price)

# Define y_valid from the valid dataset
y_valid = valid[:, 0]

# Calculate metrics
mse = mean_squared_error(y_valid, preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_valid, preds)
r2 = r2_score(y_valid, preds)

print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'R2: {r2}\n')

# For plotting
train_df = stock_lstm[:990]
valid_df = stock_lstm[990:].copy()
valid_df['Predictions'] = preds
plt.plot(train_df['Close'], label='Train')
plt.plot(valid_df['Close'], label='Actual')
plt.plot(valid_df['Predictions'], label='Predicted')
plt.legend()
plt.title(f'Predicted close price vs actual close price on {bank_stock_name}')
plt.show()

# Save model and scaler
Function_model.save_keras_model(stock_model_dir, "LSTM", model)
Function_model.update_model_loss(stock_model_dir, "LSTM", rmse, mae, r2)

scaler_path = os.path.join(stock_model_dir, "LSTM_scaler.pkl")
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f'Model and scaler saved to {stock_model_dir}')


910/910 - 29s - 32ms/step - loss: 0.0043
Fitting Model
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step


NameError: name 'root_mean_squared_error' is not defined